#### Computation of SVD


For matrix $A\in \mathbf{R}^{m \times n}$ (tall, square or fat), its SVD is given by

$$A=U\Sigma V^T=\sum_{i=1}^r\sigma_i u_i v_i^T$$

where
* $A\in \mathbf{R}^{m \times n}$, $\text{rank}(A)=r$
* $U\in \mathbf{R}^{m \times r}$, $U^TU=I$
* $V\in \mathbf{R}^{n \times r}$, $V^TV=I$
* $\Sigma =\text{diag}(\sigma_1, \cdots, \sigma_r)$, $\sigma_1 \geq\cdots\geq \sigma_r > 0$

In practice, a two phase approach is often used

* #1 Turn $A$ into a bidiagonal matrix $B$
* #2 Do SVD on $B$

#### Phase one

The idea of phase one is to apply alternately Householder reflector on the left and on the right

$$\begin{align*}A:\begin{bmatrix} \times & \times & \times \\\times & \times & \times\\\times & \times & \times\\\times & \times & \times\end{bmatrix} &\rightarrow U_1^TA\rightarrow \begin{bmatrix} \times & \times & \times \\ 0& \times & \times\\ 0 & \times & \times\\ 0 & \times & \times\end{bmatrix}\rightarrow Q_1^TAV_1\rightarrow\begin{bmatrix} \times & \times & 0 \\ 0& \times & \times\\ 0 & \times & \times\\ 0 & \times & \times\end{bmatrix}\rightarrow Q_2^TQ_1^TAV_1\rightarrow\begin{bmatrix} \times & \times & 0 \\ 0& \times & \times\\ 0 & 0 & \times\\ 0 & 0 & \times\end{bmatrix}\\&
\rightarrow Q_3^TQ_2^TQ_1^TAV_1\rightarrow \begin{bmatrix} \times & \times & 0 \\ 0& \times & \times\\ 0 & 0 & \times\\ 0 & 0 & 0\end{bmatrix}\end{align*}$$

We cannot turn $A$ into diagonal as right multiplication of Householder reflector would destroy the column just modified

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [ ]:
def bidiagonalization(A):
    m, n = A.shape
    U_T = np.identity(m)
    B = A.copy()
    V_T = np.identity(n)

    for i in range(min(m, n)):
        if i < m:
            x = B[i:, i]
            if not np.allclose(x[1:], 0):
                v = x.copy()
                sgn = np.sign(v[0]) if v[0] != 0 else 1.0
                v[0] += sgn * np.linalg.norm(x)
                v /= np.linalg.norm(v)
                # Householder from left
                # Same as Householder for QR
                B[i:, i:] -= 2 * np.outer(v, v) @ B[i:, i:]
                U_T[i:, :] -= 2 * np.outer(v, v) @ U_T[i:, :]

        if i < n - 2:
            x = B[i, i+1:]
            if not np.allclose(x[1:], 0):
                v = x.copy()
                sgn = np.sign(v[0]) if v[0] != 0 else 1.0
                v[0] += sgn * np.linalg.norm(x)
                v /= np.linalg.norm(v)
                # Householder from right
                # B^T = (I-vv^T)B^T, so B = B - B @ (vv^T)
                # [i+1:, i:] in B^T will be modified, which is [i:, i+1:] in B
                B[i:, i+1:] -= 2 * B[i:, i+1:] @ np.outer(v, v)
                # BV, same as V^TB^T, V^T = V^T - (vv^T) @ V^T
                # [i+1:, :] in V^T will be modified, same as Householder from left
                V_T[i+1:, :] -= 2 * np.outer(v, v) @ V_T[i+1:, :]

    # We need U and V_T
    return U_T.T, B, V_T

In [ ]:
np.random.seed(42)

A = np.random.rand(6, 3)
U, B, V_T = bidiagonalization(A)

print("U\n", U)
print("\nBidiagonal B\n", B)
print("\nV\n", V_T.T)

print("\nOrthogonality of U\n", U.T @ U)
print("\nOrthogonality of V\n", V_T @ V_T.T)

A_reconstructed = U @ B @ V_T
print("\nOriginal A:\n", A)
print("\nReconstructed A:\n", A_reconstructed)
print("\nReconstruction error:", np.linalg.norm(A - A_reconstructed))

U
 [[-0.2848  0.5490 -0.3409 -0.5904 -0.1877 -0.3429]
 [-0.4552 -0.2261 -0.2326  0.2383 -0.7540  0.2497]
 [-0.0442  0.6377 -0.2695  0.7035  0.1513  0.0314]
 [-0.5383  0.1637  0.7706  0.1275 -0.0111 -0.2704]
 [-0.6329 -0.3378 -0.3575  0.0004  0.5978  0.0122]
 [-0.1394  0.3162  0.1881 -0.2890  0.1260  0.8636]]

Bidiagonal B
 [[-1.3153  1.1643  0.0000]
 [ 0.0000 -1.4048 -0.4371]
 [-0.0000  0.0000  0.6834]
 [-0.0000 -0.0000  0.0000]
 [ 0.0000 -0.0000  0.0000]
 [-0.0000  0.0000  0.0000]]

V
 [[ 1.0000  0.0000  0.0000]
 [ 0.0000 -0.4878 -0.8730]
 [ 0.0000 -0.8730  0.4878]]

Orthogonality of U
 [[ 1.0000 -0.0000  0.0000 -0.0000 -0.0000 -0.0000]
 [-0.0000  1.0000 -0.0000  0.0000 -0.0000  0.0000]
 [ 0.0000 -0.0000  1.0000  0.0000 -0.0000  0.0000]
 [-0.0000  0.0000  0.0000  1.0000  0.0000 -0.0000]
 [-0.0000 -0.0000 -0.0000  0.0000  1.0000 -0.0000]
 [-0.0000  0.0000  0.0000 -0.0000 -0.0000  1.0000]]

Orthogonality of V
 [[ 1.0000  0.0000  0.0000]
 [ 0.0000  1.0000  0.0000]
 [ 0.0000  0.0000  1.00

#### Phase two (TBA)